In [6]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-164276
Azure region: southcentralus
Subscription id: 976ee174-3882-4721-b90a-b5fef6b72f24
Resource group: aml-quickstarts-164276


In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
#choose a name for my compute cluster
cpu_cluster_name = "my-cpu-cluster"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("An existing cluster has been found, please use it.")
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)
print(cpu_cluster.get_status().serialize())

InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Resizing', 'allocationStateTransitionTime': '2021-11-23T12:03:06.105000+00:00', 'errors': None, 'creationTime': '2021-11-23T12:03:05.324224+00:00', 'modifiedTime': '2021-11-23T12:03:21.471712+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [10]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": uniform(0.1,1),
    "--max_iter": choice(10,20,50,100,150,200,250)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1,delay_evaluation=4)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory="./", compute_target=cpu_cluster, vm_size="STANDARD_D2_V2", entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.

hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=25,
    max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
#my_experiment = Experiment(workspace=ws, name='hyperparametertuning')
my_hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(my_hyperdrive_run).show()
my_hyperdrive_run.get_status()
my_hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_e549aae1-9b79-4e91-a687-62758afcd4d6
Web View: https://ml.azure.com/runs/HD_e549aae1-9b79-4e91-a687-62758afcd4d6?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-164265/workspaces/quick-starts-ws-164265&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-11-23T08:05:32.698884][API][INFO]Experiment created<END>\n""<START>[2021-11-23T08:05:33.581069][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-11-23T08:05:33.754324][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_e549aae1-9b79-4e91-a687-62758afcd4d6
Web View: https://ml.azure.com/runs/HD_e549aae1-9b79-4e91-a687-62758afcd4d6?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-164265/workspaces/quick-starts-ws-164265&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_e549aae1-9b79-4e91-a687-62758afcd4d6',
 'target': 'my-cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-11-23T08:05:32.447738Z',
 'endTimeUtc': '2021-11-23T08:22:04.831826Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '38a308ae-1239-4b92-bcd0-30ffd6c06fb6',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1056-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.34.0',
  'score': '0.9089529590288316',
  'best_child_run_id': 'HD_e549aae1-9b79-4e91-a687-62758afcd4d6_21',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg164265.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_e549aae1-9b79-4e91-a687-62758afcd4d6/azureml-logs/hyperdrive.txt

KeyError: 'log_files'

In [7]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
my_hyperdrive_best_run = my_hyperdrive_run.get_best_run_by_primary_metric()

print("Details of my best run: ", my_hyperdrive_best_run.get_details(),"/n")
print("File names of my best run: ", my_hyperdrive_best_run.get_file_names(), "/n")
print("Metrics of my best run: ", my_hyperdrive_best_run.get_metrics())

my_model = my_hyperdrive_best_run.register_model(model_name="register_my_hyperdrive_best_run_log",
model_path="./", tags={"Method" : "HyperDrive"},
properties={"Accuracy": my_hyperdrive_best_run.get_metrics()["Accuracy"]})


Details of my best run:  {'runId': 'HD_e549aae1-9b79-4e91-a687-62758afcd4d6_21', 'target': 'my-cpu-cluster', 'status': 'Completed', 'startTimeUtc': '2021-11-23T08:17:50.013473Z', 'endTimeUtc': '2021-11-23T08:19:05.385649Z', 'services': {}, 'warnings': [{'message': 'This run might be using a new job runtime with improved performance and error reporting. The logs from your script are in user_logs/std_log.txt. Please let us know if you run into any issues, and if you would like to opt-out, please add the environment variable AZUREML_COMPUTE_USE_COMMON_RUNTIME to the environment variables section of the job and set its value to the string "false"'}], 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': '38a308ae-1239-4b92-bcd0-30ffd6c06fb6', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath

In [2]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
web_link = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'

ds = TabularDatasetFactory.from_delimited_files(web_link)

In [4]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd
from azureml.core import Dataset, Datastore, Workspace

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

concat_data = pd.concat([x,y], axis=1)

my_train_data, my_test_data = train_test_split(concat_data, test_size=0.2, random_state=66)

ws = Workspace.from_config()

my_datastore = ws.get_default_datastore()

final_training_data = Dataset.Tabular.register_pandas_dataframe(my_train_data, my_datastore, "data_from_pd_to_tab", show_progress=True)

Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/902e595a-e5f5-4a33-b32e-59332957b828/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [10]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=final_training_data,
    label_column_name='y',
    n_cross_validations=6,
    compute_target=cpu_cluster,
    enable_early_stopping=True)

In [11]:
# Submit your automl run

### YOUR CODE HERE ###
automl_experiment = Experiment(ws, name="my_automl_regression")
my_remote_run = exp.submit(config=automl_config,show_output=True)

Submitting remote run.
No run_configuration provided, running on my-cpu-cluster with default configuration
Running on remote compute: my-cpu-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_3b46e1de-f278-4233-ade4-fecf87fd459b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+-------------------------

In [1]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
from pprint import pprint

# Print model to print the hyperparameters from each automl training pipeline step
def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0]+ ' _ ')
        elif hasattr(step[1], '_base_learners') and hasattr(step[1], '_meta_learner'):
            print("\nMeta Learner")
            pprint(step[1]._meta_learner)
            for estimator in step[1]._base_learners:
                print_model(estimator[1], estimator[0]+ ' _ ')
        else:
            pprint(step[1].get_params())
            print()

# Get the best model
my_automl_best_run, my_automl_fitted_model = my_remote_run.get_output()
print(my_automl_best_run)

#print_model(my_automl_fitted_model)

my_automl_best_run.get_tags()

# Save my best automl run
import joblib
joblib.dump(value=my_automl_best_run.id, filename="./best_model.joblib")

NameError: name 'my_remote_run' is not defined

In [2]:
#delete compute cluster
cpu_cluster.delete()

NameError: name 'cpu_cluster' is not defined